dans ce fichier nous allons mettre les requêtes de filtres et nettoyage des données

Import pandas

In [1]:
import pandas as pd

Import des données : tables entières

In [ ]:
# tmdb = pd.read_csv("./donnees/TMDB/tmdb_full.csv", low_memory=False)

In [ ]:
# title_akas = pd.read_csv("./donnees/IMDb/title.akas.tsv/title.akas.tsv", sep="\t", low_memory=False)

In [ ]:
# title_basics = pd.read_csv("./donnees/IMDb/title.basics.tsv/title.basics.tsv", sep="\t", low_memory=False)
# title_basics.head()

In [ ]:
# title_crew = pd.read_csv("./donnees/IMDb/title.crew.tsv/title.crew.tsv", sep="\t", low_memory=False)

In [ ]:
# title_episode = pd.read_csv("./donnees/IMDb/title.episode.tsv/title.episode.tsv", sep="\t", low_memory=False)

In [ ]:
# title_principals = pd.read_csv("./donnees/IMDb/title.principals.tsv/title.principals.tsv", sep="\t", low_memory=False)

In [ ]:
# title_ratings = pd.read_csv("./donnees/IMDb/title.ratings.tsv/title.ratings.tsv", sep="\t", low_memory=False)

In [ ]:
# name_basics = pd.read_csv("./donnees/IMDb/name.basics.tsv/name.basics.tsv", sep="\t", low_memory=False)

Filtre par genre = Action et production countries = FR

In [ ]:
tmdb_filtre = tmdb[(tmdb['genres'].str.contains('Action'))|(tmdb['production_countries'].str.contains('FR'))]

In [ ]:
tmdb_filtre.info()

Liaisons à faire :
TMDB - Title_crew - title_principals - name_basics


In [ ]:
table_films_select = pd.merge(tmdb_filtre, 
                              title_crew,
                              how="inner", 
                              left_on = "imdb_id",
                              right_on = "tconst"
                              )

In [ ]:
table_films_select.info()

In [ ]:
table_films_select = table_films_select.drop(['backdrop_path','homepage','video','production_companies_name','production_companies_country'],axis=1)

In [ ]:
table_films_select = pd.merge(table_films_select, 
                              title_principals,
                              how="left", 
                              left_on = "tconst",
                              right_on = "tconst"
                              )

In [ ]:
table_films_select.head()

In [ ]:
table_films_select = table_films_select.drop(['ordering','job'],axis=1)

In [ ]:
table_films_select = pd.merge(table_films_select, 
                              name_basics,
                              how="left", 
                              left_on = "nconst",
                              right_on = "nconst"
                              )

In [ ]:
table_films_select.head()

In [ ]:
table_films_select = table_films_select.drop(['birthYear','deathYear','primaryProfession'],axis=1)

In [ ]:
table_films_select.info()

In [ ]:
table_films_select.columns

Index(['adult', 'budget', 'genres', 'id', 'imdb_id', 'original_language',
       'original_title', 'overview', 'popularity', 'poster_path',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'tconst', 'directors', 'writers', 'nconst', 'category',
       'characters', 'primaryName', 'knownForTitles'],
      dtype='object')

In [ ]:
table_films_select.to_csv('./donnees/table_films_select.csv', sep="\t")

In [ ]:
table_films_select.info()

On part du csv créer table_films_select.csv

In [4]:
films_tmdb_imdb = pd.read_csv("./donnees/table_films_select.csv", sep="\t", low_memory=False)

In [5]:
films_tmdb_imdb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 770236 entries, 0 to 770235
Data columns (total 29 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   Unnamed: 0            770236 non-null  int64  
 1   adult                 770236 non-null  bool   
 2   budget                770236 non-null  int64  
 3   genres                770236 non-null  object 
 4   id                    770236 non-null  int64  
 5   imdb_id               770236 non-null  object 
 6   original_language     770236 non-null  object 
 7   original_title        770236 non-null  object 
 8   overview              732422 non-null  object 
 9   popularity            770236 non-null  float64
 10  poster_path           729654 non-null  object 
 11  production_countries  770236 non-null  object 
 12  release_date          762336 non-null  object 
 13  revenue               770236 non-null  int64  
 14  runtime               770236 non-null  int64  
 15  

In [6]:
films_tmdb_imdb = films_tmdb_imdb.drop('Unnamed: 0',axis=1)

In [7]:
films_tmdb_imdb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 770236 entries, 0 to 770235
Data columns (total 28 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   adult                 770236 non-null  bool   
 1   budget                770236 non-null  int64  
 2   genres                770236 non-null  object 
 3   id                    770236 non-null  int64  
 4   imdb_id               770236 non-null  object 
 5   original_language     770236 non-null  object 
 6   original_title        770236 non-null  object 
 7   overview              732422 non-null  object 
 8   popularity            770236 non-null  float64
 9   poster_path           729654 non-null  object 
 10  production_countries  770236 non-null  object 
 11  release_date          762336 non-null  object 
 12  revenue               770236 non-null  int64  
 13  runtime               770236 non-null  int64  
 14  spoken_languages      770236 non-null  object 
 15  

In [ ]:
films_tmdb_imdb.describe()

In [8]:
films_tmdb_imdb['release_date']=pd.to_datetime(films_tmdb_imdb['release_date'])

In [ ]:
films_tmdb_imdb.info()

On drop tagline car rarement l'info

In [9]:
films_tmdb_imdb = films_tmdb_imdb.drop('tagline', axis=1)

On ne garde que ce qui ont un résumé

In [10]:
films_tmdb_imdb.dropna(subset = ['overview'], inplace=True)

In [ ]:
films_tmdb_imdb.info()

On ne garde que ce qui a un poster

In [11]:
films_tmdb_imdb.dropna(subset = ['poster_path'], inplace=True)

In [ ]:
films_tmdb_imdb.isna().sum()

In [12]:
films_tmdb_imdb.dropna(subset = ['release_date'], inplace=True)

In [ ]:
films_tmdb_imdb.isna().sum()

On garde que si info sur personne = nconst, category, ...

In [13]:
films_tmdb_imdb.dropna(subset = ['nconst','category','characters','primaryName','knownForTitles'], inplace=True)

In [ ]:
films_tmdb_imdb.isna().sum()

In [ ]:
films_tmdb_imdb.info()

On drop tconst car fait doublon avec imdb_id

In [14]:
films_tmdb_imdb[['tconst','imdb_id']]

,tconst,imdb_id
0,tt0055747,tt0055747
1,tt0055747,tt0055747
2,tt0055747,tt0055747
3,tt0055747,tt0055747
4,tt0055747,tt0055747
...,...,...
770231,tt17352384,tt17352384
770232,tt17352384,tt17352384
770233,tt17352384,tt17352384
770234,tt17352384,tt17352384


In [15]:
films_tmdb_imdb = films_tmdb_imdb.drop('tconst', axis=1)

In [ ]:
films_tmdb_imdb.info()

On renomme id  en id_tmdb pour clarifier

In [16]:
films_tmdb_imdb = films_tmdb_imdb.rename(columns={'id':'id_tmdb'})

on va filtrer selon le vote average basé sur un nombre de vote plutôt  que popularity on ne sait pas trop comment c'est calculé et ce n'est pas cohérent avec vote_average

In [17]:
films_tmdb_imdb[['vote_average','popularity']].value_counts(sort=True)


vote_average  popularity
0.000         0.600         25684
              1.400          3689
5.000         0.600          3440
0.000         0.840          3334
6.000         0.600          3332
                            ...  
6.800         1.294             1
4.100         1.031             1
6.672         3.988             1
4.900         0.854             1
5.900         1.820             1
Name: count, Length: 29444, dtype: int64

Max de vote_average= 10 = les films sont notés sur 10

In [18]:
films_tmdb_imdb['vote_average'].max()

10.0

on prend les films avec votete_average supérieur à 5

In [19]:
films_tmdb_imdb_sup5 = films_tmdb_imdb[films_tmdb_imdb['vote_average']>5]

In [ ]:
films_tmdb_imdb_sup5.info()

filtre selon date de sortie

In [20]:
from datetime import datetime

on ajoute une colonne année

In [21]:
films_tmdb_imdb_sup5['year'] = films_tmdb_imdb_sup5['release_date'].dt.year

C:\Users\LOTUS\AppData\Local\Temp\ipykernel_27140\2621779178.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  films_tmdb_imdb_sup5['year'] = films_tmdb_imdb_sup5['release_date'].dt.year


In [22]:
films_tmdb_imdb_sup5.info()

<class 'pandas.core.frame.DataFrame'>
Index: 462885 entries, 0 to 770134
Data columns (total 27 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   adult                 462885 non-null  bool          
 1   budget                462885 non-null  int64         
 2   genres                462885 non-null  object        
 3   id_tmdb               462885 non-null  int64         
 4   imdb_id               462885 non-null  object        
 5   original_language     462885 non-null  object        
 6   original_title        462885 non-null  object        
 7   overview              462885 non-null  object        
 8   popularity            462885 non-null  float64       
 9   poster_path           462885 non-null  object        
 10  production_countries  462885 non-null  object        
 11  release_date          462885 non-null  datetime64[ns]
 12  revenue               462885 non-null  int64         
 13  runt

In [23]:
films_tmdb_imdb_sup5['year'].value_counts(sort=True)

year
2019    17487
2017    17399
2022    16683
2018    16511
2016    15588
        ...  
1898       20
1894        8
1892        7
1874        4
1878        1
Name: count, Length: 133, dtype: int64

On va garder les années après 2000 inclus

In [24]:
films_tmdb_imdb_sup5_2000 = films_tmdb_imdb_sup5[films_tmdb_imdb_sup5['year']> 1999]

In [25]:
films_tmdb_imdb_sup5_2000.info()

<class 'pandas.core.frame.DataFrame'>
Index: 272237 entries, 70 to 770046
Data columns (total 27 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   adult                 272237 non-null  bool          
 1   budget                272237 non-null  int64         
 2   genres                272237 non-null  object        
 3   id_tmdb               272237 non-null  int64         
 4   imdb_id               272237 non-null  object        
 5   original_language     272237 non-null  object        
 6   original_title        272237 non-null  object        
 7   overview              272237 non-null  object        
 8   popularity            272237 non-null  float64       
 9   poster_path           272237 non-null  object        
 10  production_countries  272237 non-null  object        
 11  release_date          272237 non-null  datetime64[ns]
 12  revenue               272237 non-null  int64         
 13  run

Nombre de films sélectionnés : 

In [26]:
len(films_tmdb_imdb_sup5_2000['id_tmdb'].unique())

14672

durée des films 

In [ ]:
liste = list(films_tmdb_imdb_sup5_2000['runtime'].unique())
liste.sort()
liste

on prend les films plus longs que 60 minutes

In [28]:
films_tmdb_imdb_sup5_2000_60 = films_tmdb_imdb_sup5_2000[films_tmdb_imdb_sup5_2000['runtime']>=60]

In [ ]:
films_tmdb_imdb_sup5_2000_60.info()

Nombre de films qui durent plus de 60 minutes inclues sortis à partir de 2000 et qui ont une note supérieure à 5.

In [30]:
len(films_tmdb_imdb_sup5_2000_60['id_tmdb'].unique())

12771

On prend les films qui sont sortis (Released) et on enlève les films In Production => en fait il n'y a pas de films en production

In [31]:
films_tmdb_imdb_sup5_2000_60[films_tmdb_imdb_sup5_2000_60['status'] == 'In Production']

,adult,budget,genres,id_tmdb,imdb_id,original_language,original_title,overview,popularity,poster_path,...,vote_average,vote_count,directors,writers,nconst,category,characters,primaryName,knownForTitles,year


In [32]:
films_tmdb_imdb_sup5_2000_60_released = films_tmdb_imdb_sup5_2000_60[films_tmdb_imdb_sup5_2000_60['status'] == 'Released']

In [ ]:
films_tmdb_imdb_sup5_2000_60_released.info()

In [33]:
len(films_tmdb_imdb_sup5_2000_60_released['original_title'].unique())


12644

In [34]:
len(films_tmdb_imdb_sup5_2000_60_released['imdb_id'].unique())

12771

Je regarde si les original_title apparaissent plusieurs fois = OUI => mais c'est normal soit plusieurs films avec même titre soit plusieurs acteurs...

In [ ]:
films_tmdb_imdb_sup5_2000_60_released['original_title'].value_counts()

In [ ]:
films_tmdb_imdb_sup5_2000_60_released[films_tmdb_imdb_sup5_2000_60_released['original_title']=="Charlie's Angels"]

In [ ]:
len(test_films['id_tmdb'].unique())

12771

In [ ]:
len(test_films['title'].unique())

12458

On regarde dans title_akas la région de la version du titre et on prend quand France   
On met na=False pour ignorer les valeurs NaN sinon on a une erreur

In [ ]:
# title_akas_FR = title_akas[title_akas['region'].str.contains('FR', na=False)]

In [ ]:
# title_akas_FR.info()

In [ ]:
# title_akas_FR_reduit = title_akas_FR[['titleId','title','region','language']]
# title_akas_FR_reduit.info()

In [ ]:
# films_tmdb_imdb_sup5_2000_60_released_regFR = pd.merge(films_tmdb_imdb_sup5_2000_60_released,
#                                                        title_akas_FR_reduit,
#                                                        how='inner',
#                                                        left_on='imdb_id',
#                                                        right_on='titleId')

In [36]:
films_select = films_tmdb_imdb_sup5_2000_60_released[['adult','budget','genres','id_tmdb','imdb_id','original_language','original_title','original_title','overview','popularity','poster_path','production_countries','release_date' ,'revenue','runtime','spoken_languages','status','title' ,'vote_average','vote_count','directors','writers'                        
                        
                        ]]


In [37]:
films_select.duplicated().sum()

233308

In [40]:
films_select = films_select.drop_duplicates()

In [41]:
films_select.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12771 entries, 70 to 769907
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   adult                 12771 non-null  bool          
 1   budget                12771 non-null  int64         
 2   genres                12771 non-null  object        
 3   id_tmdb               12771 non-null  int64         
 4   imdb_id               12771 non-null  object        
 5   original_language     12771 non-null  object        
 6   original_title        12771 non-null  object        
 7   original_title        12771 non-null  object        
 8   overview              12771 non-null  object        
 9   popularity            12771 non-null  float64       
 10  poster_path           12771 non-null  object        
 11  production_countries  12771 non-null  object        
 12  release_date          12771 non-null  datetime64[ns]
 13  revenue            

In [42]:
films_select.to_csv('./donnees/films_selectionnes.csv', sep="\t")

In [46]:
films_tmdb_imdb_sup5_2000_60_released.to_csv('./donnees/films_tmdb_imdb_sup5_2000_60_released.csv', sep="\t")